# Session 2: 확률과 분포 (Integrated)

## 🎯 학습 목표
- **Deep Theory**: 확률의 3가지 공리와 조건부 확률, 베이즈 정리의 수식적 의미를 이해합니다.
- **Practice**: 몬테카를로 시뮬레이션과 Bigram 확률을 직접 구현해봅니다.
- **Concepts**: 확률 변수와 주요 분포(정규, 로그 정규, Zipf)의 특성을 파악합니다.
- **Experiment**: 몬티 홀 문제를 통해 조건부 확률의 직관을 검증합니다.

---

# Part 1. 📐 Deep Theory: 확률의 본질

## 1. 확률의 기초 (Axioms of Probability)

표본 공간(Sample Space) $\Omega$에서 사건(Event) $E$가 일어날 확률 $P(E)$는 다음 3가지 공리를 만족해야 합니다 (Kolmogorov axioms).

1.  **Non-negativity**: 모든 사건 $E$에 대해 $P(E) \ge 0$.
2.  **Normalization**: 전체 표본 공간의 확률은 1이다. $P(\Omega) = 1$.
3.  **Additivity**: 서로 배반인 사건들($E_1, E_2, ...$ where $E_i \cap E_j = \emptyset$)의 합집합 확률은 각 확률의 합과 같다.
    $$ P(\bigcup_{i=1}^{\infty} E_i) = \sum_{i=1}^{\infty} P(E_i) $$

## 2. 조건부 확률 (Conditional Probability)

사건 $B$가 일어났다는 전제 하에 사건 $A$가 일어날 확률입니다.

$$ P(A|B) = \frac{P(A \cap B)}{P(B)} $$

> **Insight**: $P(B)$로 나누어준다는 것은, 전체 표본 공간을 $\Omega$에서 $B$로 **'축소(Normalize)'** 시킨다는 의미입니다.

## 3. 베이즈 정리 (Bayes' Theorem)

조건부 확률의 정의를 뒤집어서, $P(A|B)$와 $P(B|A)$의 관계를 설명합니다.

$$ P(A|B) = \frac{P(B|A)P(A)}{P(B)} $$

- $P(A)$: 사전 확률 (Prior) - 증거를 보기 전의 믿음
- $P(B|A)$: 우도 (Likelihood) - 가설 $A$가 맞을 때 증거 $B$가 관측될 확률
- $P(A|B)$: 사후 확률 (Posterior) - 증거 $B$를 본 후 갱신된 믿음

## 4. PDF와 CDF

- **PDF (Probability Density Function, $f(x)$)**: 연속 확률 변수의 특정 구간에서의 밀도.
- **CDF (Cumulative Distribution Function, $F(x)$)**: $-\infty$부터 $x$까지의 확률 누적.
- 관계: $F(x) = \int_{-\infty}^{x} f(t) dt$, 반대로 $f(x) = \frac{d}{dx}F(x)$

---
# Part 2. 💻 Manual Implementation: 시뮬레이션과 확률

### 미션 1: 몬테카를로(Monte Carlo) 시뮬레이션으로 $\pi$ 구하기
사각형 안에 원을 그리고, 무작위로 점을 찍었을 때 원 안에 들어갈 확률은 면적비($\pi/4$)와 같습니다.
이를 이용해 $\pi$를 추정해보세요.

In [ ]:
import random

def estimate_pi(n_samples):
    inside_circle = 0
    
    for _ in range(n_samples):
        x = random.random() # 0 ~ 1 사이
        y = random.random()
        
        # 원점(0,0)에서의 거리가 1 이하이면 원 안
        if x**2 + y**2 <= 1:
            inside_circle += 1
            
    return (inside_circle / n_samples) * 4

# 실험
for n in [100, 1000, 10000, 100000]:
    print(f"N={n}, Pi={estimate_pi(n):.5f}")

### 미션 2: Bigram 확률 계산기
간단한 코퍼스를 주고, $P(\text{word}|\text{prev_word})$를 계산하는 함수를 만듭니다.

In [ ]:
corpus = "I am a boy and I am a student".split()

def calc_bigram_prob(prev, word, corpus):
    # TODO: 조건부 확률 P(word|prev) 계산
    # P(word|prev) = Count(prev, word) / Count(prev)
    
    bigram_count = 0
    prev_count = 0
    
    for i in range(len(corpus) - 1):
        if corpus[i] == prev:
            prev_count += 1
            if corpus[i+1] == word:
                bigram_count += 1
                
    if prev_count == 0:
        return 0
    return bigram_count / prev_count

print(f"P(a|am) = {calc_bigram_prob('am', 'a', corpus)}")
print(f"P(student|a) = {calc_bigram_prob('a', 'student', corpus)}")

---
# Part 3. 📖 Concepts & Practice: 분포의 세계

## Zipf's Law (멱 법칙)
자연어에서 단어의 빈도는 순위(Rank)에 반비례합니다.
$$ P(x) \propto x^{-\alpha} $$
대부분의 단어는 아주 조금 나오고, 소수의 단어가 엄청나게 많이 나옵니다.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Zipf 분포 생성 (a=1.5)
zipf_data = np.random.zipf(a=1.5, size=1000)

# 상위 50개만 시각화
from collections import Counter
counts = Counter(zipf_data)
sorted_counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)[:50]

ranks = range(1, len(sorted_counts)+1)
freqs = [x[1] for x in sorted_counts]

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.bar(ranks, freqs)
plt.title("Word Frequency (Linear Scale)")
plt.xlabel("Rank")
plt.ylabel("Frequency")

plt.subplot(1, 2, 2)
plt.loglog(ranks, freqs, marker='o')
plt.title("Word Frequency (Log-Log Scale)")
plt.xlabel("Log Rank")
plt.ylabel("Log Frequency")
plt.show()

---
# Part 4. 🧪 Experiment: 몬티 홀 문제 (Monty Hall Problem)

**상황**: 문 3개 중 하나에 차가 있고, 나머지 둘엔 염소가 있습니다. 하나를 고르면, 사회자가 남은 문 중 염소가 있는 문을 열어줍니다. 바꾸는 게 유리할까요?

- **Stay**: 처음에 고른 문을 유지.
- **Switch**: 다른 문으로 바꿈.

직관적으로는 반반(50%) 같지만, 조건부 확률로 계산하면 **바꾸는 게 2배 유리(66%)**합니다. 시뮬레이션으로 검증해봅시다.

In [ ]:
def monty_hall(strategy='stay', n_trials=1000):
    wins = 0
    
    for _ in range(n_trials):
        doors = [0, 0, 1] # 1 is Car
        random.shuffle(doors)
        
        choice = random.randint(0, 2)
        
        # 사회자가 염소 문을 연다 (내가 고른 것 제외, 차가 있는 것 제외)
        # 남은 문 중 차가 있는 문을 선택하면 승리
        
        if strategy == 'stay':
            if doors[choice] == 1:
                wins += 1
        elif strategy == 'switch':
            # 내가 고르지 않은 나머지 두 문 중 하나는 염소임. 
            # 사회자가 그 염소 문을 열어줌.
            # 내가 바꾼 문에 차가 있으려면? -> 애초에 내가 염소를 골랐으면 됨.
            if doors[choice] == 0:
                wins += 1
                
    return wins / n_trials

print(f"Win Rate (Stay): {monty_hall('stay') * 100:.2f}%")
print(f"Win Rate (Switch): {monty_hall('switch') * 100:.2f}%")

> **Discussion**: 코드로 돌려보니 정말로 Switch가 약 66% 승률이 나옵니다. 조건부 확률(새로운 정보가 추가되었을 때의 확률)의 힘입니다.